In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Flask,send_file
from flask import jsonify
from flask_cors import CORS
from flask import Response
from sklearn.metrics import mean_squared_error
import base64
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.style.use('seaborn')
import seaborn as sns
import numpy as np
from io import StringIO 
import pandas as pd
%matplotlib inline
import base64
import json 
from json import dumps
from flask import request
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import statsmodels.api as sm
import io
import random
import pickle
app = Flask(__name__)

CORS(app)
@app.route('/image')
def serve_image():
    CleanDataset=pd.read_csv('New_Final.csv')
    columns = CleanDataset.iloc[:, 14:19].join(CleanDataset.iloc[:, 22:31])
    sizes = columns.sum(axis=0)
    plt.ion()
    plt.clf()

    plt.figure(figsize=(8, 5))
    sizes.plot.pie()
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    response = Response(buf.getvalue(), mimetype='image/png')
    plt.ioff()
    return response

def Dailydata(weekly):
    weekly["date"] = pd.to_datetime(weekly["date"])
    weekly_summary =weekly.resample('w' ,on='date').agg({ 'p1' :'sum', 'p2' : 'sum', 'p3' :'sum', 'p4' : 'sum', 'p5' :'sum', 'p6' : 'sum', 'p7' :'sum','p8' :'sum','p9' :'sum','p10' :'sum','p11' :'sum','p12' :'sum','p13' :'sum','p14' :'sum','p15' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    weekly_summary['date']=weekly_summary.index
    return weekly_summary
def Weeklydata(weekly):
    weekly["date"] = pd.to_datetime(weekly["date"])
    weekly_summary =weekly.resample('w' ,on='date').agg({ 'p1' :'sum', 'p2' : 'sum', 'p3' :'sum', 'p4' : 'sum', 'p5' :'sum', 'p6' : 'sum', 'p7' :'sum','p8' :'sum','p9' :'sum','p10' :'sum','p11' :'sum','p12' :'sum','p13' :'sum','p14' :'sum','p15' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    weekly_summary['date']=weekly_summary.index
    return weekly_summary
def Monthlydata(weekly):
    monthly = pd.DataFrame()

    weekly["date"] = pd.to_datetime(weekly["date"])
    Monthly_summary =weekly.resample('M' ,on='date').agg({ 'p1' :'sum', 'p2' : 'sum', 'p3' :'sum', 'p4' : 'sum', 'p5' :'sum', 'p6' : 'sum', 'p7' :'sum','p8' :'sum','p9' :'sum','p10' :'sum','p11' :'sum','p12' :'sum','p13' :'sum','p14' :'sum','p15' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    Monthly_summary['date']=Monthly_summary.index
    return Monthly_summary
def Yearlydata(weekly):
    weekly["date"] = pd.to_datetime(weekly["date"])
    yearly_summary =weekly.resample('Y' ,on='date').agg({ 'p1' :'sum', 'p2' : 'sum', 'p3' :'sum', 'p4' : 'sum', 'p5' :'sum', 'p6' : 'sum', 'p7' :'sum','p8' :'sum','p9' :'sum','p10' :'sum','p11' :'sum','p12' :'sum','p13' :'sum','p14' :'sum','p15' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    yearly_summary['date']=yearly_summary.index
    return yearly_summary

@app.route('/GetGraphs',methods=['GET', 'POST'])
def dashboardgraphs():
    dailydataset=pd.read_csv('New_Final.csv')
    if request.method == 'POST':
        content = request.json
        startDate=content["startDate"]
        endDate=content["endDate"]
        types=content["type"]
        if(types=='Weekly'):
             dailydataset=Dailydata(dailydataset)
        elif(types=='Monthly'):
             dailydataset=Monthlydata(dailydataset)
        else:
             dailydataset=Yearlydata(dailydataset)
        plotgraph(dailydataset,startDate,endDate)
        with open("plot.png", "rb") as img_file:
            my_img = base64.b64encode(img_file.read())
            base64_string = my_img .decode('utf-8')
    else:
        dailydataset=Dailydata(CleanDataset)
        plotgraph(dailydataset,'2017-06-01','2017-07-13')
        with open("plot.png", "rb") as img_file:
            my_img = base64.b64encode(img_file.read())
            base64_string = my_img .decode('utf-8')
    return jsonify(base64_string)


@app.route('/data', methods=['GET', 'POST'])
def data():
    z=process_data()
    print(z)
    return z
    

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

    
def plotgraph(dataset,start_date,end_date):
    plt.ion()
    plt.clf()

    dataset['date'] = pd.to_datetime(dataset['date'])
    df2 = dataset.query('date>= @start_date and date<=@end_date')
    Date = df2['date']
    p1 = df2['p1']
    p2 = df2['p2']
    p3 = df2['p3']
    p4 = df2['p4']
    p5 = df2['p5']
    p6 = df2['p6']
    p7 = df2['p7']
    p8 = df2['p8']
    p9 = df2['p9']
    p10 = df2['p10']
    p11 = df2['p11']
    p12 = df2['p12']
    p13 = df2['p13']
    p14 = df2['p14']
    p15 = df2['p15']
    
    plt.figure(figsize=(8, 5))
    plt.rcParams['figure.figsize'] = (14,9)
    plt.rcParams['date.converter'] = 'concise'
    plt.rcParams['date.interval_multiples'] = True
    plt.plot(Date, p1, color='tab:purple', label='p1')
    plt.plot(Date, p2, color='tab:blue', label='p2')
    plt.plot(Date, p3, color='tab:pink', label='p3')
    plt.plot(Date, p4, color='tab:red', label='p4')
    plt.plot(Date, p5, color='tab:brown', label='p5')
    plt.plot(Date, p6, color='tab:gray', label='p6')
    plt.plot(Date, p7, color='tab:cyan', label='p7')
    plt.plot(Date, p8, color='tab:olive', label='p8')
    plt.plot(Date, p9, color='tab:purple', label='p9')
    plt.plot(Date, p10, color='tab:blue', label='p10')
    plt.plot(Date, p11, color='tab:pink', label='p11')
    plt.plot(Date, p12, color='tab:red', label='p12')
    plt.plot(Date, p13, color='tab:brown', label='p13')
    plt.plot(Date, p14, color='tab:gray', label='p14')
    plt.plot(Date, p15, color='tab:cyan', label='p15')
    plt.legend()
    plt.title('Sales ')
    plt.xlabel('Date')
    plt.ylabel('Sales numbers')
    plt.xticks(rotation=45)
    plt.savefig('plot.png')
    plt.ioff()

@app.route('/Applygradientdecent',methods=['GET', 'POST'])    
def PredictValueAccordingTomodel():
    if request.method == 'POST':
        content = json.loads(request.data)
        Type= (content["type"])
        Target= content["target"]
        input_data = pd.DataFrame({
        'Temperature': content["Temperature"],
        'Dew Point': [float(content["Dew_Point"])],
        'Humidity': [float(content["Humidity"])],
        'Wind Speed': [float(content["Wind_Speed"])],
        
        'Pressure': [float(content["Pressure"])],
        'IntCondition': [float(content["Condition_int"])],
        'IntSeason': [float(content["Seasons"])],
            'Event': [int(content["Event"])],
        
         })
        if Type == "Daily": 
            with open("daily/"+Target + ".pkl", 'rb') as model_file:
                loaded_model = pickle.load(model_file)
        elif Type == "Weekly":
            with open("daily/"+Target + "_weekly.pkl", 'rb') as model_file:
                loaded_model = pickle.load(model_file)
        elif Type=="Monthly":
            with open("daily/"+Target + "_monthly.pkl", 'rb') as model_file:
                loaded_model = pickle.load(model_file)
        elif Type=="Quaterly":
            with open("daily/"+Target + "_qtr.pkl", 'rb') as model_file:
                loaded_model = pickle.load(model_file)
        prediction = loaded_model.predict(input_data)
        if prediction < 0:
            prediction = abs(prediction)
        resut={
            'Predictions':str(prediction)
        }
        json_result = json.dumps(resut)
        return json_result


if __name__=="__main__":
    app.run(host='0.0.0.0', port=105)
    

C:\Users\Ibrahim\AppData\Local\Temp\ipykernel_6304\3624667139.py:22: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:105
 * Running on http://192.168.43.26:105
Press CTRL+C to quit
127.0.0.1 - - [18/Sep/2023 23:40:46] "POST /Applygradientdecent HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2023 23:44:26] "GET /image HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2023 00:15:35] "POST /Applygradientdecent HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2023 00:16:40] "POST /Applygradientdecent HTTP/1.1" 200 -
